# Exercise 12: Sequence-to-Sequence modelling using transformers

In this exercise, we train a very small transformer architecture on an algorithmic toy task.

We then 'introspect' the trained model (i.e. inspect its activations for different data samples), to find out what algorithm it learned and when it (does not) achieve the correct predictions.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch
from torch.utils.data import Dataset, random_split
from collections import Counter
import numpy as np
import math
import matplotlib.pyplot as plt

# Part 1: Creating a sequence-to-sequence task

In language modelling, the primary application of transformers, we often aim to map a sequence of words to a second sequence of words.
Think about translating a sentence from one language to another, or ChatGPT creating an answer given a question by you.
To make clear what the model sees when you translate a sentence, consider this example:
$$
[\textrm{The, dog, chases, the, cat, .}] \to [\textrm{Der, Hund, jagt, die, Katze, .}]
$$
or
$$
[\textrm{What, is, 2, times, 2, ?}] \to [\textrm{The, answer, is, 4, .}]
$$

In the machine learning context, a single word is called a "token". The set of all possible tokens is called "alphabet". Current open source models like BERT that can be downloaded on the internet have about ~30,000 tokens.

In this exercise, we focus on a very specific sequence-to-sequence task, namely predicting the histogram of a sequence.
Here, our tokens are just integer numbers from one to $T$, and an example is the following:
$$
[21,15,21,4,4,4,17] \to [2,1,2,3,3,3,1]
$$

More formally,assume you are given the sequence of tokens $\mathbf s = [s_1,s_2,\cdots,s_d]$ of length $d$ (`seq_len` in the code) where $s_i \in [1,\cdots,T]$, we want to predict its histogram, the sequence $\mathbf h^s = \mathrm{hist}(\mathbf s)$.
At the position of the token $s_i$ in the original sequence, we find the number of times it occured in $S$, i.e.
$$h^s_i = |\{j =1,...,d| s_j = s_i \}|.$$


In the following we provide the dataset to load this data for a given length of the sequence $d$ (`seq_len`) and number of tokens $T$.


In [ ]:
def hist(s):
  c = Counter(s)
  c = {w: c[w] for w in c}
  return [c[w] for w in s]

class HistogramDataset(Dataset):
    def __init__(self, seq_len, T, n_samples,seed=42):
        self.seq_len = seq_len
        self.T = T
        self.n_samples = n_samples
        rs = np.random.RandomState(seed)
        self.X = rs.randint(0, T, (n_samples, seq_len))
        self.X = np.unique(self.X, axis=0)
        self.y = np.empty_like(self.X)
        for i in range(n_samples):
          self.y[i] = hist(self.X[i])

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx],dtype=torch.long), torch.tensor(self.y[idx],dtype=torch.long)

#### Question 1.1
Generate a dataset for the histogram with a $T$ and $d$ of your choice. Print two samples from the dataset.

In [ ]:
... # your code here

Ellipsis

#### Question  1.2
Given $T, d > 0$ and a sequence $\mathbf s$ given that is generated from the data. What is the largest possible value that any $h^s_i$ can achieve? What is the smallest value?

Verify that this is the case for an example from the dataset.

In [ ]:
... # your code here

Ellipsis

In the following code, we define the dataset and a train and validation split that we will then use to train and evaluate out transformer model that learns the $\mathrm{hist}$ function.

In [ ]:
# Define the length, maximum value, and number of samples
seq_len = 10
T = 15
num_samples = 10000
n_classes = seq_len+1

# Create the dataset
dataset = HistogramDataset(seq_len, T, num_samples)

# Split the dataset into train, test, and validation sets
train_ratio, val_ratio = 0.7, 0.3

train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Part 2: Building A Transformer - First layer: Word Embeddings

In the following we define a "mini" transformer. The "mini" is to note that it does not have all the features as the transformer architecture when it was introduced, and to note that it would never be able to achieve SOTA results on common language tasks due to its limited capacity.

There are many details and specialities to the architecture and training modalities of transformers in the wild that are beyond the scope of this course, and that have also changed rapidly over the last few of years.

The focus in this exercise is on developing a deep understanding of the heart of the transformer architecture that has been a constant, the dot product attention layer that you also discussed in the lecture.

We go through the architecture of the transformer step-by-step for an example data batch and then create the complete model.

#### Question 2.1
Verify that the first batch of samples of input tensor x and the output tensor y have the correct shapes (what are the shapes you would expect)?

In [ ]:
# take the first batch of data from the training data
B = 7 # batch size
x, y =  next(iter(DataLoader(train_dataset, batch_size=B)))

In [ ]:
... # your code here

Ellipsis

#### Question 2.2

In the sequence to sequence task, our input and our output is categorial (an element of a fixed set of choices). While you have seen categorial outputs in classification, we did not yet work with categorial inputs.

So how can we input the token 'cat' to the transformer? The solution is to use one-hot encodings, i.e. vectors $v$ of length $T$. To represent the word 'cat' using a vector $v_{\mathrm{cat}}$ we set the elements of the vector to zero except at position $i_{\mathrm{cat}}$ where we set it to $1$.

That means, the first layer of a model that processes such categorial input has to be of size $W_{\mathrm{emb}} \in \mathbb{R}^{T\times r}$, were $r$ is the model dimension for further processing (called `model_dim` in the code). When the remainder of the model architecture is added and the model is trained on a given task, the weights $W_{\mathrm{emb}}$ are trained along with other parameters of the model.

Often, one attributes a special name to this very first processing layer, it is called the `word embedding`. By multiplying $W_{\mathrm{emb}}$ with a one-hot vector, one selects row of the matrix. Therefore, every row represents a single token, you can think of the row $[W_{\mathrm{emb}}]_{i_{\mathrm{cat}}}$ as a representation of cat in a $r$-dimensional space.

In torch, the process of one-hot encoding and multiplying by the embedding matrix is already implemented in the `nn.Embedding` module, which we now use to create the embedding for our histogram task.

- Write out in latex, what the one-hot encoding for the token '3' would look like, if overall there are 10 different tokens in my alphabet.


**Your answer here:**

#### Question 2.2

Have a look at the documentation of the `nn.Embedding` module in torch.
You can use it in a similar fashion you used the torch sequential model.
You first create an instance, e.g. `model = nn.Sequential(...,...)`. Afterwards you apply the module to some input by calling `output = model(input)`.

Create an embedding module from `nn.Embedding`. So far there is no need to train the parameters within, we will just look at what it does.

Apply the randomly initialized embedding to the batch `x` of size `B x seq_len` and create an embedding `e` of size `B x seq_len x model_dim`.

Verify by example that the same token at different positions in the input `x` is mapped to to the same embedding vector.



In [ ]:
model_dim = 32
... # your code here

## Part 3: Building a Transformer: The Dot-Product Attention Layer

The dot product attention is defined as follows for the $Q, K, V \in \mathbb R^{r \times r}$ and $X \in \mathbb R^{d \times r}$, so that a row in the matrix is the embedded token $\mathbf x_i = [x_{i1},x_{i2},\cdots,x_{ir}] \in \mathbb R^r$.
The outcome of the attention layer at a specific token is then the linear combination of all other tokens, transformed acccording to the value matrix $V$.

$$
\text{AttentionLayer}(X; Q, K, V)_{i} =  A_{i}V \mathbf X \in \mathbb R^r
$$

where $A_{i}$ is an $d$-dimensional vector respresenting a probability distribution over all other tokens. The value $a_{ij}$ represents how much attention is payed from token at position $i$ to position $j$.
This attention value is  computed as a query type funtion:

$$
A_i = \text{att_probs}(X; Q, K)_{i} = [a_{i1},a_{i2},\cdots,a_{id}] = \text{softmax}\left(\frac{(\mathbf x_iQ) (XK)^T}{\sqrt{r}}\right) \in \mathbb R^d
$$

We transform the embedding $\mathbf x_i$ into a query of dimension $r$ via the matrix $Q$, all the other tokens including itself get transformed to keys using $K$. The comparison between the $d$ keys and the query from token $i$ is via the dot-product (think cosine similarity, so a high value when the vectors representing keys and queries point in the same direction ~represent similar concepts).
Finally, the softmax transforms this vector of cosine similarities of $d$ values into a probability distribution, so that every value is between 0 and 1 and they all together sum to 1.

#### Question 3.1

Below, we give you a skeleton for a cusom nn.Module, that we will later use in a sequential module to built a complete transformer architecture.

Implement the attention mechanism in the forward function.

Hint: Do this together with question 3.2, so your debugging is easier.

In [ ]:
class DotProductAttention(nn.Module):
  def __init__(self, d):
    super(DotProductAttention, self).__init__()

    self.model_dim = model_dim

    self.Q = nn.Parameter(torch.empty(model_dim, model_dim))
    self.K = nn.Parameter(torch.empty(model_dim, model_dim))
    self.V = nn.Parameter(torch.empty(model_dim, model_dim))

    nn.init.kaiming_uniform_(self.Q.T, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.K.T, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.V.T, a=math.sqrt(5))

    self.attn_probs = None # we want to readout the attention matrix later

  def forward(self, x):
    # x: B x seq_len x model_dim
    Q = self.Q # model_dim x model_dim
    K = self.K # model_dim x model_dim
    V = self.V # model_dim x model_dim

    # precompute important quantities
    Qx =  ... # B x seq_len x model_dim
    Kx =  ... # B x seq_len x model_dim
    Vx =  ... # B x seq_len x model_dim

    # compute the attention scores
    attn_scores = ... # B x seq_len x seq_len

    # apply the softmax
    attn_probs = ...  # B x seq_len x seq_len
    self.attn_probs = attn_probs # save attention matrix for later

    # do a linear combination of the values
    x = ... # B x seq_len x model_dim
    return x


#### Question 3.2

Apply and test the dot product to the attention you coded to the embeddings `e` you created previously.

Hint: Remember that it is used like any other torch module used before, you first instantiate it and then you apply it to a vector by calling it on some input.

In [ ]:
... # your code here

(torch.Size([7, 10, 32]), torch.Size([7, 10, 32]), torch.Size([7, 10, 10]))

## Part 4: Building a Transformer: Final Architecture

We now successfully mixed the tokens in a given sequence, by mixing we mean: at the position of token $i$ there is now a linear combination of all other tokens in the sequence.

Now to fit a function like the histogram task, we still have to process the information that got collected in an intelligent way.

To do so, we apply an MLP module to the outcomes of the attention layer: A one hidden layer MLP with $h$ hidden neurons and relu activation. Since we want to predict tokens from our target alphabet, the output should correspond to the possible number of classes (in our case they range from 0 to L for the histogram task).
We can use the `nn.Sequential` module to combine all necessary steps.

#### Question 4.1
Complete the model below and verify that it successfully tranforms the input batch $x$. What is the shape of the outcome? Does it make sense?

In [ ]:
p = 128
n_classes = seq_len+1
transformer = nn.Sequential(nn.Embedding(T, model_dim),
                            DotProductAttention(model_dim),
                            nn.LayerNorm(model_dim), # we add layernorm to make training faster, but you can ignore what it does (but if you remove it you might have to wait a bit until training converges)
                            ... ) # TODO: add a fully connected network with h hidden neurons and relu activations

output = transformer(x)
output.shape

#### Question 4.2
How do you extract a prediction for the output sequence from output?

In [ ]:
... # your code here

Ellipsis

# Part 5: Training the Transformer

You can have a look at how the training works, but it is not the objective of this exercise. Essentially we do supervised learning on the output sequence using the cross-entropy loss you know from classification with neural networks.

Overall, executing the training should not last longer then 4min.


#### Question 5.1

Paste the code for you transformer architecture from 4.1 and then run the training loop.

In [ ]:
lr = 0.001
n_epochs = 100

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

transformer = nn.Sequential(...) # TODO: paste your solution from 4.2

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-9)

train_losses = []
val_losses = []
val_acc = []

for epoch in range(n_epochs):

        epoch_loss = 0.0

        for X, y in train_dataloader:
            optimizer.zero_grad()

            output = transformer(X)
            loss = criterion(output.contiguous().view(-1, n_classes), y.contiguous().view(-1))

            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()

        epoch_loss /= len(train_dataloader)
        train_losses.append(epoch_loss)

        # Evaluate on the test set every epoch
        with torch.no_grad():
            val_loss = 0.0
            acc = 0.0
            for X, y in val_dataloader:
                output = transformer(X)
                pred = output.argmax(axis=-1)
                loss = criterion(output.view(-1,n_classes), y.view(-1))
                acc += torch.mean((pred.view(-1)==y.view(-1)).float()).item()
                val_loss+=loss.item()
            val_loss /= len(val_dataloader)
            acc /= len(val_dataloader)

            val_losses.append(val_loss)
            val_acc.append(acc)

        if epoch % 10 == 0:
          print(f'[Epoch {epoch:02}] Train loss = {epoch_loss:.5f} :: Val loss {val_loss:.5f} :: Val accuracy {acc*100:.2f}')

**Hint:** At this point, you should be able to obtain a validation accuracy around 97-99 %. If not, it might help to just restart the training another time.

#### Question 5.2

Check the loss curves below, and the validation accuracy, does the transformer successfully fit the histogram task?

In [ ]:
plt.plot(train_losses,label='test')
plt.plot(val_losses,label='val')
plt.xlabel('epochs')
plt.legend(title='loss');

# Part 6: Model Introspection
In the final section of this exercise, we want to understand the algorithm the transformer learned, by probing predictions for specific corner-cases and looking at some of its internals.


In [ ]:
def highlight_cell(x,y,color):
    # Given a coordinate (x,y), highlight the corresponding cell using a colored frame in the ac
    # after having called imshow already
    rect = plt.Rectangle((x-.5, y-.5), 1,1, fill=False,color=color,lw=2)
    plt.gca().add_patch(rect)
    return rect

def visualize_attention_matrix(x):
  x_ = torch.tensor(x,dtype=torch.long)
  transformer(x_)
  A = transformer[1].attn_probs
  plt.imshow(A.detach().numpy(),vmin=0,cmap='winter')
  for i, x_i in enumerate(x):
    for j, x_j in enumerate(x):
      if x_i == x_j:
        highlight_cell(i,j, color='red')
  plt.colorbar()
  plt.xticks(np.arange(len(x)), x)
  plt.yticks(np.arange(len(x)), x)
  plt.xlabel('Input sequence')
  plt.ylabel('Input sequence')
  plt.title(f'Attention matrix')

#### 6.1 What is the attention layer used for?
Visualize the attention layer probabilities for some examples and highlight the cells that represent the attention onto tokens with the same token type using the given code.

- Do you have a guess on what algorithm the model is learning?
- Why is the attention layer useful?
- Do you expect it to work for shorter/longer sequences?

In [ ]:
x = [1,1,3,2,3,3,0,0,1,1]
visualize_attention_matrix(x)

In [ ]:
... # test more examples

Ellipsis

#### 6.2 Does the transformer truly generalize?

Since you now have a hypothesis of what your transformer learned, check its predictions carefully. Use the function we provide to predict a single sequence.

- Is the validation error good?
- Does it generalize to longer sequences?
- Go back to the Histgram dataset and look at how the data is generated. What sequences are unlikely to be generated. Does it generalize those low probability input samples?
- Does it generalize to unknown tokens?
- Would you have expected the answers to the above questions, given the validation error you saw?

In [ ]:
def predict_sequence(x):
  pred = transformer.forward(torch.tensor(x,dtype=torch.long)).argmax(axis=-1).detach().numpy()
  y = hist(x)
  return list(pred), np.all(y == pred)

In [ ]:
predict_sequence([1,4,3,2,3,3,0,0,4])

In [ ]:
... # try more examples

Ellipsis

# Bonus Part 7: What else?

We kept this exercise rather concise, but of course with more interesting architectures there is more to explore. You also saw that even such an "easy" task as the histogram task is not trivially solved by a simple transformer.

However, there are many ways to improve, tweaks to make, and architectural desicions to make.

If you want a full example of a transformer architecture from scratch, we recommend the following online tutorial: ["Transformer from scratch"](https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb) that implements the [original transformer architecture](https://arxiv.org/pdf/1706.03762.pdf).

Also feel free to think about the following, which are active or recent research areas in machine learning:
- Where is the computational bottleneck for longer sentences? How can one circumvent it?
- How does one make positional information enter the model to idenify positional information?
- What are the training strategies when I want to predict the next word only, and not a complete sequence?

Also note: It is not so difficult to use pretrained language models! Instead of training a complete model from scratch, if you want to use a model in application domains where there is true understanding of text, one can use pre-trained models available for free online (e.g. [hugging face](https://huggingface.co/)) and adapt them as you are doing in the current assignment .